# Application de détection automatique des faux billets

Ce notebook présente une application de prédiction permettant de déterminer si un billet est authentique ou falsifié à partir de ses caractéristiques géométriques.

Le modèle utilisé est une **régression logistique**, entraînée sur des données préalablement nettoyées et imputées.  
L’objectif est de pouvoir tester l’algorithme en conditions réelles à partir d’un fichier de production fourni lors de la soutenance.

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

## Chargement des données et entraînement du modèle

Le modèle est entraîné à partir du jeu de données historique des billets, contenant les mesures géométriques ainsi que la variable cible `is_genuine`.

In [5]:
# Chargement du dataset d'entraînement
df = pd.read_csv("billets.csv", sep=";")

# Variables explicatives
features = [
    "diagonal",
    "height_left",
    "height_right",
    "margin_up",
    "length"
]

X = df[features]
y = df["is_genuine"].astype(int)

# Séparation train / test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# Entraînement du modèle final
model_log = LogisticRegression(max_iter=1000)
model_log.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

## Fonction de prédiction d’un billet

La fonction suivante permet de prédire si un billet est authentique ou falsifié à partir de ses caractéristiques géométriques.
Elle retourne également la probabilité associée à la prédiction.

In [8]:
def predict_bill(diagonal, height_left, height_right, margin_up, length):
    """
    Prédit l'authenticité d'un billet à partir de ses mesures géométriques.
    """
    data = pd.DataFrame([{
        "diagonal": diagonal,
        "height_left": height_left,
        "height_right": height_right,
        "margin_up": margin_up,
        "length": length
    }])

    proba = model_log.predict_proba(data)[0][1]
    prediction = model_log.predict(data)[0]

    if prediction == 1:
        return f"Billet authentique (probabilité : {proba:.2%})"
    else:
        return f"Billet falsifié (probabilité : {1 - proba:.2%})"

In [10]:
predict_bill(
    diagonal=171.8,
    height_left=104.2,
    height_right=104.1,
    margin_up=3.2,
    length=113.1
)

'Billet authentique (probabilité : 92.41%)'

In [22]:
# Chargement du fichier de production
df_prod = pd.read_csv("billets_production.csv")

# Colonnes utilisées par le modèle
features = [
    "diagonal",
    "height_left",
    "height_right",
    "margin_up",
    "length"
]

# Prédictions
df_prod["proba_authentique"] = model_log.predict_proba(df_prod[features])[:, 1]
df_prod["prediction"] = model_log.predict(df_prod[features])

# Résultat lisible
df_prod["resultat"] = df_prod["prediction"].apply(
    lambda x: "Authentique" if x == 1 else "Falsifié"
)

# Sélection finale des colonnes à afficher
df_prod[["id", "resultat", "proba_authentique"]]

,id,resultat,proba_authentique
0,A_1,Falsifié,0.024531
1,A_2,Falsifié,0.066659
2,A_3,Falsifié,0.001248
3,A_4,Authentique,0.898359
4,A_5,Authentique,0.998483
